In [1]:
import sys
sys.path.append("../")
import pandas as pd
import openpyxl as oxl

from utils.prompts import render
from utils.llm_client import LLMClient
from utils.logging_utils import log_llm_call
from utils.router import pick_model,should_use_reasoning_model
from IPython.display import display, Markdown
from utils.token_utils import count_messages_tokens,fit_within_context

In [2]:
# Load the text file
df = pd.read_csv(
    "../data/raw/incidents.txt",
    sep="|",
    engine="python"
)

# Clean column names (remove spaces)
df.columns = df.columns.str.strip()

# Extract only the Message column
messages = df["Message"].astype(str).tolist()

print(messages)


[' "Thirsty but safe on roof. Water level stable." ', ' "Diabetic, missed dose yesterday. Feeling faint."', ' "Water approaching neck level. Child is crying."']


In [3]:
user_input_constraint = """
You are a “Budget Keeper” tasked with managing user messages for token efficiency. 

Rules:
1. The input is a single string that contains **all messages combined together**.
2. Count the number of tokens in the combined string using a tokenization method.
3. If the combined input exceeds 150 tokens:
    - Do NOT process the content.
    - Print only: "BLOCKED/TRUNCATED" as a success check.
4. If the combined input is 150 tokens or less:
    - Keep it as-is and output the entire combined string.
5. Do NOT modify messages under the token limit.
6. Process the combined input as a single unit.
7. Output format:
    - If ≤150 tokens: "Message OK: <entire combined messages>"
    - If >150 tokens: "BLOCKED/TRUNCATED"
8. Be concise. Do not add extra explanations.

Example:
Input: "Message 1. Message 2. Message 3." (all messages combined into one string)  
Output: "Message OK: Message 1. Message 2. Message 3."

Input: "Very long combined messages exceeding 200 tokens..."  
Output: "BLOCKED/TRUNCATED"
"""


In [4]:
# Combine all messages into one string
all_messages_text = "\n".join(messages)

full_prompt = f"{user_input_constraint}\n\nMessages:\n{all_messages_text}"

messages_response = [{"role": "user", "content": full_prompt}]

# Count tokens for the combined messages
token = count_messages_tokens(messages=messages_response, provider="groq", model="llama-3.1-8b-instant")

# check if message exceeds 150 tokens
if token['estimated_total'] > 150:
    print("BLOCKED/TRUNCATED")
else:
    print("Message OK:")
    print(all_messages_text)


BLOCKED/TRUNCATED
